<h1 align='center'> Curso de Estadística Bayesiana<br> Análisis Exploratorio de Modelos Bayesianos</h1> 

<h3>Autor</h3>

1. Alvaro Mauricio Montenegro Díaz, ammontenegrod@unal.edu.co
2. Daniel Mauricio Montenegro Reyes, dextronomo@gmail.com 

<h3>Fork</h3>

<h3>Referencias</h3>


1. Gelman, A., Carlin, J., Stern, H., and Rubin, D. *Bayesian Data Analysis*, Chapman & Hall/CRC, 2000
2. https://statmodeling.stat.columbia.edu/2014/01/21/everything-need-know-bayesian-statistics-learned-eight-schools/
3. 
[PyStan: Getting Started](https://pystan.readthedocs.io/en/latest/getting_started.html)
4. [Manual de usuario de Stan, versión 2.22](https://mc-stan.org/docs/2_22/stan-users-guide/index.html)
5. [ArviZ: Exploratory analysis of Bayesian models](https://arviz-devs.github.io/arviz/)

<h2> 1. Introducción</h2>

Se introducen los elementos esenciales del análisis explortorio de  Modelos Bayesianos. Usaremos la librería [Arviz](https://arviz-devs.github.io/arviz/)

En este cuaderno usamos el ejemplo de 8 escuelas (eight schools) que utilizamos en el [primer ejemplo de Stan en este curso](Stan_Ejemplo_1_eight_schools.ipynb)





<h2> 2. EL modelo estadístico para este  tutorial</h2>


Supongamos que tenemos observaciones $[y_n |x_n], n=1,\ldots,N$, y asumamos el modelo Bayesiano. 

**Modelo con efectos aleatorios**

- Estima los hiperparámetros $\mu$ and $\tau$
- Predice los efectos aleatorios $\eta_i$


$$
\begin{align}
y_i &\sim \mathcal{N}(\theta_i,\sigma_i^2), \text{ known } \sigma_i^2\\
\theta_i & = \mu + \tau \times \eta_i \\
\eta_i &\sim \mathcal{N}(0, 1)\\
\mu &\sim \mathcal{N}(0, 5)\\
\tau &\sim \mathcal{Cauchy}(0, 5)1_{\tau>0.0}
\end{align}
$$



<h3>  Los datos</h3>

Los datos corresponden a la estimación (estandarizada) de un puntaje realizada en las ocho escuelas observadas.



|School |Estim. Treatment Effect | Estim. Stand. Error |
|---|---|---|
|A |28| 15|
|B |8 |10|
|C| -3| 16|
|D| 7| 11|
|E| -1 |9|
|F| 1| 11|
|G |18| 10|
|H| 12| 18|

In [1]:
# importamos las librerías requieridas
import pystan
import numpy as np
# Se requiren instaladas las libreria matplotlib and scipy pra los gráficos 


In [53]:
# los datos
J = 8
y = np.array([28.,  8., -3.,  7., -1.,  1., 18., 12.])
sigma = np.array([15., 10., 16., 11.,  9., 11., 10., 18.])
schools = np.array(['Choate', 'Deerfield', 'Phillips Andover', 'Phillips Exeter',
                    'Hotchkiss', 'Lawrenceville', "St. Paul's", 'Mt. Hermon'])

schools_dat = {'J': J,
               'y': y,
               'sigma': sigma}
schools_dat 

{'J': 8,
 'y': array([28.,  8., -3.,  7., -1.,  1., 18., 12.]),
 'sigma': array([15., 10., 16., 11.,  9., 11., 10., 18.])}

In [4]:
print(type(schools_dat))
print(schools_dat['y'])

<class 'dict'>
[28, 8, -3, 7, -1, 1, 18, 12]


<h2> 3. Implementación del modelo estadístico en Stan</h2>

In [9]:
schools_code = """
data {
    int<lower=0> J;
    real y[J];
    real<lower=0> sigma[J];
}

parameters {
    real mu;
    real<lower=0> tau;
    real eta[J];
}

transformed parameters {
    real theta[J];
    for (j in 1:J)
        theta[j] = mu + tau * eta[j];
}

model {
    mu ~ normal(0, 5);
    tau ~ cauchy(0, 5);
    eta ~ normal(0, 1);
    y ~ normal(theta, sigma);
}

generated quantities {
    vector[J] log_lik;
    vector[J] y_hat;
    for (j in 1:J) {
        log_lik[j] = normal_lpdf(y[j] | theta[j], sigma[j]);
        y_hat[j] = normal_rng(theta[j], sigma[j]);
    }
}
"""


In [10]:
print(schools_code)


data {
    int<lower=0> J;
    real y[J];
    real<lower=0> sigma[J];
}

parameters {
    real mu;
    real<lower=0> tau;
    real eta[J];
}

transformed parameters {
    real theta[J];
    for (j in 1:J)
        theta[j] = mu + tau * eta[j];
}

model {
    mu ~ normal(0, 5);
    tau ~ cauchy(0, 5);
    eta ~ normal(0, 1);
    y ~ normal(theta, sigma);
}

generated quantities {
    vector[J] log_lik;
    vector[J] y_hat;
    for (j in 1:J) {
        log_lik[j] = normal_lpdf(y[j] | theta[j], sigma[j]);
        y_hat[j] = normal_rng(theta[j], sigma[j]);
    }
}



En este código los parámetros que serán muestreados son $\mu$ (*mu*), $\tau$ (*tau*), que es una desviación estándard y los $\eta_i$ (*eta*) que son los efectos aleatorios en el modelo. Las medias $\theta_i$ de las escuelas son calculados. Es decir, son funcionales de los parámetros.

Hemos asumido las distribuciones a priori para $\mu\sim \mathcal{N}(0,5)$,  $\tau\sim \mathcal{Cauchy}(0,5)1_{\tau>0.0}$ y $\eta_i \sim \mathcal{N}(0,1)$.

Adicionalmente hemos incluido la sección *generated quantities* con el propósito calcular valores útiles para la validación del modelo.

El objeto *log_lik* se usa para calcular la log-verosimilitud (a partir de la función *normal_lpdf*). El objeto *y_hat* se usa para generar réplicas de las observaciones (partir de la función *normal_rng*). Esta es **la distribución predictiva posterior**.



<h2> 4. Compilación del Modelo</h2>

In [11]:
# compile the  model
sm = pystan.StanModel(model_code=schools_code)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_e7c3cbd6ff9855fd1a960862c5979009 NOW.


In [12]:
# extract the samples
fit = sm.sampling(data=schools_dat, iter=1000, chains=4)

In [13]:
fit

Inference for Stan model: anon_model_e7c3cbd6ff9855fd1a960862c5979009.
4 chains, each with iter=1000; warmup=500; thin=1; 
post-warmup draws per chain=500, total post-warmup draws=2000.

             mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
mu            4.4    0.08   3.34  -2.47   2.11   4.47   6.73  10.84   1639    1.0
tau          3.53    0.09   3.22    0.1   1.25   2.67   4.79  12.15   1285    1.0
eta[1]        0.3    0.02    1.0   -1.8  -0.34   0.31   0.96   2.23   1776    1.0
eta[2]        0.1    0.02   0.94  -1.77  -0.51    0.1   0.75   1.91   2186    1.0
eta[3]      -0.07    0.02   0.93   -1.8  -0.72  -0.08   0.59   1.75   1888    1.0
eta[4]       0.07    0.02   0.95  -1.84  -0.57    0.1   0.71   1.88   1978    1.0
eta[5]      -0.18    0.02   0.97  -2.12  -0.82   -0.2   0.46   1.78   1779    1.0
eta[6]      -0.06    0.02   0.93  -1.87   -0.7  -0.05   0.55   1.81   2079    1.0
eta[7]       0.33    0.02   0.92  -1.57  -0.29   0.37   0.97   2.04   1688 

<h2> 5. Inferencia. InferenceData (ArviZ) </h2>

Paar seguir esta parte del tutorial por favor instale ArviZ.

Desde la linea de comandos: 

*pip install arviz*

o mejor use

*conda install -c conda-forge arviz*

[InferenceData](https://arviz-devs.github.io/arviz/notebooks/InferenceDataCookbook.html) es la principal estrutura de datos de ArviZ, la herramienta recomenda para nuestros análisis con Stan. 

In [54]:
import arviz as az
import numpy as np
import pandas as pd
import xarray

az.style.use('arviz-darkgrid')

xarray.set_options(display_style="html");

stan_data = az.from_pystan(
    posterior=fit,
    posterior_predictive='y_hat',
    observed_data=['y'],
    log_likelihood={'y': 'log_lik'},
    coords={'school': schools },
    dims={
        'theta': ['school'],
        'y': ['school'],
        'log_lik': ['school'],
        'y_hat': ['school'],
        'eta': ['school']
    }
)

In [31]:
stan_data

Inference data with groups:
	> posterior
	> sample_stats
	> log_likelihood
	> posterior_predictive
	> observed_data

In [32]:
stan_data.posterior

<xarray.Dataset>
Dimensions:  (chain: 4, draw: 500, school: 8)
Coordinates:
  * chain    (chain) int64 0 1 2 3
  * draw     (draw) int64 0 1 2 3 4 5 6 7 8 ... 492 493 494 495 496 497 498 499
  * school   (school) int64 0 1 2 3 4 5 6 7
Data variables:
    mu       (chain, draw) float64 8.028 7.016 5.655 ... -2.143 2.674 6.577
    tau      (chain, draw) float64 8.345 4.084 1.786 ... 0.004203 0.01841
    eta      (chain, draw, school) float64 -0.231 0.473 ... 0.5981 0.08466
    theta    (chain, draw, school) float64 6.101 11.97 1.016 ... 6.588 6.579
Attributes:
    created_at:                 2020-03-30T00:01:26.821719
    arviz_version:              0.7.0
    inference_library:          pystan
    inference_library_version:  2.19.0.0

In [22]:
stan_data.sample_stats

<xarray.Dataset>
Dimensions:      (chain: 4, draw: 500)
Coordinates:
  * chain        (chain) int64 0 1 2 3
  * draw         (draw) int64 0 1 2 3 4 5 6 7 ... 493 494 495 496 497 498 499
Data variables:
    accept_stat  (chain, draw) float64 0.9843 0.7251 0.7547 ... 0.9754 0.9426
    stepsize     (chain, draw) float64 0.4391 0.4391 0.4391 ... 0.3713 0.3713
    treedepth    (chain, draw) int64 3 3 3 3 4 3 3 4 3 3 ... 3 3 3 4 3 4 4 4 3 4
    n_leapfrog   (chain, draw) int64 7 7 7 7 31 15 7 23 ... 7 15 7 15 23 15 7 15
    diverging    (chain, draw) bool False False False ... False False False
    energy       (chain, draw) float64 6.24 11.9 10.02 ... 17.6 19.16 17.47
    lp           (chain, draw) float64 -4.309 -3.686 -3.694 ... -13.9 -11.66
Attributes:
    created_at:                 2020-03-29T23:44:37.993593
    arviz_version:              0.7.0
    inference_library:          pystan
    inference_library_version:  2.19.0.0

In [23]:
stan_data.log_likelihood

<xarray.Dataset>
Dimensions:  (chain: 4, draw: 500, school: 8)
Coordinates:
  * chain    (chain) int64 0 1 2 3
  * draw     (draw) int64 0 1 2 3 4 5 6 7 8 ... 492 493 494 495 496 497 498 499
  * school   (school) int64 0 1 2 3 4 5 6 7
Data variables:
    y        (chain, draw, school) float64 -4.693 -3.301 ... -3.873 -3.855
Attributes:
    created_at:                 2020-03-29T23:44:38.001326
    arviz_version:              0.7.0
    inference_library:          pystan
    inference_library_version:  2.19.0.0

In [24]:
stan_data.posterior_predictive

<xarray.Dataset>
Dimensions:  (chain: 4, draw: 500, school: 8)
Coordinates:
  * chain    (chain) int64 0 1 2 3
  * draw     (draw) int64 0 1 2 3 4 5 6 7 8 ... 492 493 494 495 496 497 498 499
  * school   (school) int64 0 1 2 3 4 5 6 7
Data variables:
    y_hat    (chain, draw, school) float64 25.42 17.5 -11.47 ... 13.29 -3.322
Attributes:
    created_at:                 2020-03-29T23:44:38.005125
    arviz_version:              0.7.0
    inference_library:          pystan
    inference_library_version:  2.19.0.0

In [25]:
stan_data.observed_data

<xarray.Dataset>
Dimensions:  (school: 8)
Coordinates:
  * school   (school) int64 0 1 2 3 4 5 6 7
Data variables:
    y        (school) float64 28.0 8.0 -3.0 7.0 -1.0 1.0 18.0 12.0
Attributes:
    created_at:                 2020-03-29T23:44:37.971742
    arviz_version:              0.7.0
    inference_library:          pystan
    inference_library_version:  2.19.0.0

In [27]:
stan_data.constant_data

AttributeError: 'InferenceData' object has no attribute 'constant_data'

In [28]:
stan_data.prior

AttributeError: 'InferenceData' object has no attribute 'prior'

In [ ]:
az.plot_density(stan_data, var_names=['mu', 'tau']);

In [ ]:
az.plot_autocorr(stan_data, var_names=("tau","mu"))

In [ ]:
#size = (2, 10)
az.plot_forest(stan_data, var_names=("tau","mu", "theta"))

In [ ]:
az.plot_trace(stan_data, var_names=("tau","mu"))

In [ ]:
az.plot_mcse(stan_data)

In [ ]:
az.plot_pair(stan_data, coords={'school': ['Choate', 'Deerfield', 'Phillips Andover']}, divergences=True);